# Public API V2 <a class="anchor" id="notebook-head"></a>

## Введение <a class="anchor" id="introduction"></a>

Здравствуйте и добро пожаловать в ноутбук, посвящённый функциональности 
[Public API V2](https://docs.sbercloud.ru/aicloud/mlspace/concepts/api.html) 
в рамках платформы 
[ML Space](https://sbercloud.ru/ru/aicloud/mlspace).

Как и множество других платформ подобного рода, ML Space предоставляет свой 
[REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) 
с тем, чтобы пользователи могли частично или полностью автоматизировать свои пайплайны.

Разумеется это позволяет оптимизировать процесс разработки, избавившись от рутинных повторяющихся манипуляций со своими данными/моделями/etc. посредством 
[ML Space UI](https://docs.sbercloud.ru/aicloud/mlspace/index.html).

Кроме того, люди разные и кому-то просто комфортнее и проще работать через REST API.

Пример: у пользователя есть уже заготовленный скрипт, который при незначительных модификациях будет взаимодействовать с ML Space посредством REST API).

Правилом хорошего тона является не только генерация документации по REST API, но и предоставление подробно описанных примеров использования.

Данный ноутбук предназначен как раз для этого.

Здесь мы рассмотрим ряд операций, которые вы можете выполнять при помощи REST API, а именно:
- Авторизация пользователя 
- Работа с данными
    - Выгрузка содержание бакета
    - Копирование данных из S3 на NFS
    - Копирование данных из NFS на S3
- Базовые Docker-образы
    - Получение списка базовых Docker-образов
- Задачи на кластере
    - Получение списка задач
    - Запуск задачи
    - Выгрузка логов задачи
- Inference-методы
    - Получение списка сервисов
    - Получение информации по сервису
    - Создание образа для инференс-сервиса
    - Просмотр статуса сборки образа для инференс-сервиса
    - Просмотр логов сборки образа для инференс-сервиса
    - Создание деплоя на основе собранного образа
    - Отправление запроса к созданному сервису

Прежде чем приступить к непосредственному рассмотрению примеров, необходимо проверить, соблюдены ли некоторые предварительные требования, а именно:
* <span style="color:green;font-weight:bold">(НАСТОЯТЕЛЬНО РЕКОМЕНДУЕТСЯ)</span> Запуск данного ноутбука предпочтительно запускать в Jupyter Lab. Это связано с тем, что Jupyter Lab (в отличие от Jupyter Notebook) умеет красиво визуализировать JSON-структуры, с которыми мы в рамках данного ноутбука будем очень тесно соприкасаться;

* Наличие рядом с самим ноутбуком JSON-файла с необходимыми авторизационными данными от вашего аккаунта (<span style="color:red;font-weight:bold">ВНИМАНИЕ:</span> это необходимо <span style="color:red;font-weight:bold">исключительно и только</span> для удобства. По существу вы можете не создавать такого файла, главное иметь при себе логин и пароль от вашего аккаунта, чтобы иметь возможность авторизоваться);

* Наличие ```X-Api-Key```, он же ```GWAPI_KEY``` (более подробно описан [здесь](https://docs.sbercloud.ru/aicloud/mlspace/concepts/api.html#id4));

* Наличие ```X-Workspace-Id``` (используется практически во всех REST-запросах в Public API V2).

Для вашего удобства представлено содержание (в самом начале ноутбука), кроме того, в каждом последующем разделе представлена обратная ссылка на содержание.

Приятной работы!

# Содержание <a class="anchor" id="toc"></a>

* [Public API V2](#notebook-head)
    * [Введение](#introduction)
* [Все необходимые библиотеки](#all-imports-necessary)
* [Вспомогательные функции и переменные](#auxiliary-stuff)
* [Предварительные требования](#prerequisites)
    * [JSON-файл с необходимыми авторизационными данными](#prerequisites-part-1)
    * [X-Api-Key](#prerequisites-part-2)
    * [X-Workspace-Id](#prerequisites-part-3)
* [Авторизация](#authorization)
* [Работа с данными](#s3)
    * [Выгрузить содержание бакета](#s3-get-bucket-content-structure)
    * [Копировать данные из S3 на NFS](#s3-copy-to-nfs)
    * [Копировать данные из NFS на S3](#s3-copy-from-s3)
* [Базовые Docker-образы](#docker-images)
    * [Получить список базовых Docker-образов](#docker-images-get)
* [Задачи на кластере](#jobs)
    * [Получить список задач](#jobs-get)
    * [Запустить задачу](#jobs-run)
    * [Выгрузить логи задачи](#jobs-job-logs)
* [Inference-методы](#inference)
    * [Получить список сервисов](#inference-get-services-list)
    * [Получить информацию по сервису](#inference-get-service-info)
    * [Создать образ для инференс-сервиса](#inference-create-an-image)
    * [Просмотреть статус сборки образа для инференс-сервиса](#inference-image-creation-status)
    * [Просмотреть  логи сборки образа для инференс-сервиса](#inference-image-creation-logs)
    * [Создать деплой на основе собранного образа](#inference-create-service)
    * [Отправить запрос к созданному сервису](#inference-get-a-response-from-service)

# Все необходимые библиотеки ([содержание](#toc))<a class="anchor" id="all-imports-necessary"></a>

Практика показывает, что концентрация всех импортов библиотек в одной ячейке в самом начале ноутбука является наиболее предпочтительным вариантом с точки зрения удобства и чистоты кода.

Так мы здесь и поступим:

In [ ]:
import os

import json

import copy

import boto3

import requests

import client_lib

from IPython.display import JSON

# Вспомогательные функции и переменные ([содержание](#toc))<a class="anchor" id="auxiliary-stuff"></a>

In [ ]:
def wipe_out_sensitive_information(func):
    
    def recursive_information_deletion(credentials_dict):
        sensitive_info_fields = [
            "id", "gwapi-key",
            "client_id", "client_secret",
            "access_key_id", "secret_access_key",
            "endpoint_url", "bucket_name",
            "access_token", "refresh_token",
            "s3_endpoint", "aws_access_key_id",
            "aws_secret_access_key"
        ]
        
        if isinstance(credentials_dict, dict):
            for k, v in credentials_dict.items():
                if isinstance(v, dict):
                    new_v = recursive_information_deletion(v)
                    
                    credentials_dict[k] = new_v
                elif isinstance(v, str):
                    if k in sensitive_info_fields:
                        credentials_dict[k] = "[SENSITIVE INFORMATION, WIPED OUT]"
                elif isinstance(v, list):
                    for i, item in enumerate(v):
                        new_v_i = recursive_information_deletion(item)
                        
                        v[i] = new_v_i
                        
        return credentials_dict

    def wrapper(credentials_dict):
        credentials_dict_copy = copy.deepcopy(credentials_dict)
        
        credentials_dict_copy["[THIS IS A COPY]"] = True
        
        credentials_dict_copy = recursive_information_deletion(credentials_dict_copy)
        
        return_value = func(credentials_dict_copy)
        
        return return_value
    
    return wrapper

In [ ]:
JSON = wipe_out_sensitive_information(JSON)

In [ ]:
host = "https://api.aicloud.sbercloud.ru"

rest_call_template = "{}/public/v2/{}"

# Предварительные требования ([содержание](#toc))<a class="anchor" id="prerequisites"></a>

## JSON-файл с необходимыми авторизационными данными ([содержание](#toc))<a class="anchor" id="prerequisites-part-1"></a>

In [ ]:
with open("credentials_dict_example.json", "r") as f:
    credentials_dict = json.load(f)

In [ ]:
JSON(credentials_dict)

In [ ]:
with open("credentials_dict.json", "r") as f:
    credentials_dict = json.load(f)

In [ ]:
credentials = {
    "client_id": credentials_dict["workspaces"][0]["client_id"],
    "client_secret": credentials_dict["workspaces"][0]["client_secret"]
}

## X-Api-Key ([содержание](#toc))<a class="anchor" id="prerequisites-part-2"></a>

```X-Api-Key``` (он же ```GWAPI_KEY```) - это клиентский ключ доступа к API (подробнее по [ссылке](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__develop-func.html)).

Он индивидуален для каждого [воркспейса](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__workspace.html) в рамках аккаунта пользователя.

При создании нового Jupyter Server-а в окружении создается переменная ```GWAPI_KEY```, туда и записывается значение ```X-Api-Key```.

Есть как минимум три способа достать эту сущность, оба особо ничем друг от друга не отличаются.

Первый - через Python:

```python
import os
print(os.environ["GWAPI_KEY"])
```

Второй - через командную оболочку (будь то bash, sh, zsh etc.):

```bash
echo $GWAPI_KEY
```

Третий - через bash-magic команду в Jupyter Notebook/Lab:

```bash
%%bash

echo $GWAPI_KEY
```

или

```bash
! echo $GWAPI_KEY
```

Для дальнейшего использования мы воспользуемся первым способом:

## X-Workspace-Id ([содержание](#toc))<a class="anchor" id="prerequisites-part-3"></a>

```X-Workspace-Id``` - это уникальный идентификатор вашего [воркспейса](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__workspace.html), в котором вы работаете.

Более подробно вы можете почитать об этом поле в соответствующем разделе [документации](https://docs.sbercloud.ru/aicloud/mlspace/concepts/deployments__send-http-requests-to-service.html?highlight=workspace%20id#http).

# Авторизация ([содержание](#toc))<a class="anchor" id="authorization"></a>

In [ ]:
endpoint = "service_auth"

json_body = json.dumps(credentials)

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    data=json_body
)

auth_response_json = json.loads(response.text)

In [ ]:
JSON(auth_response_json)

# Работа с данными ([содержание](#toc))<a class="anchor" id="s3"></a>

## Выгрузить содержание бакета ([содержание](#toc))<a class="anchor" id="s3-get-bucket-content-structure"></a>

In [ ]:
s3 = boto3.client(
    service_name="s3",
    aws_access_key_id=credentials_dict["workspaces"][0]["access_key_id"],
    aws_secret_access_key=credentials_dict["workspaces"][0]["secret_access_key"],
    endpoint_url=credentials_dict["workspaces"][0]["endpoint_url"]
)

In [ ]:
resp = s3.list_objects(Bucket=credentials_dict["workspaces"][0]["bucket_name"])

In [ ]:
if "Contents" in resp:
    bucket_content = {}

    for item in resp["Contents"]:
        sub_folders = item["Key"].split("/")

        temp = bucket_content

        for sub_folder in sub_folders:
            if sub_folder not in temp:
                temp[sub_folder] = {}

                temp = temp[sub_folder]
            else:
                temp = temp[sub_folder]

In [ ]:
JSON(bucket_content)

## Копировать данные из S3 на NFS ([содержание](#toc))<a class="anchor" id="s3-copy-to-nfs"></a>

In [ ]:
s3.download_file(
    credentials_dict["workspaces"][0]["bucket_name"],
    "test_script.py",
    "/home/jovyan/test_script.py"
)

In [ ]:
! ls -alh /home/jovyan/ | grep test_script.py

## Копировать данные из NFS на S3 ([содержание](#toc))<a class="anchor" id="s3-copy-from-s3"></a>

In [ ]:
s3.upload_file(
    "/home/jovyan/env_dependencies_tree",
    credentials_dict["workspaces"][0]["bucket_name"],
    "env_dependencies_tree",
)

# Базовые Docker-образы ([содержание](#toc))<a class="anchor" id="docker-images"></a>

## Получить список базовых Docker-образов ([содержание](#toc))<a class="anchor" id="docker-images-get"></a>

In [ ]:
endpoint = "service/base_mt_images"

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url
)

response_json = json.loads(response.text)

In [ ]:
JSON(response_json)

# Задачи на кластере ([содержание](#toc))<a class="anchor" id="jobs"></a>

## Получить список задач ([содержание](#toc))<a class="anchor" id="jobs-get"></a>

In [ ]:
endpoint = "jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

jobs_response_json = json.loads(response.text)

In [ ]:
JSON(jobs_response_json)

## Запустить задачу ([содержание](#toc))<a class="anchor" id="jobs-run"></a>

In [ ]:
endpoint = "jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "base_image": "registry.aicloud.sbcp.ru/horovod-tf15",
    "script": "/home/jovyan/test_script.py",
    "n_workers": 2,
    "n_gpus": 2,
    "warm_cache": "True",
    "flags" : {
        "batch_size": "512",
        "model":"resnet50",
        "xla":"False"
    }
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Выгрузить логи задачи ([содержание](#toc))<a class="anchor" id="jobs-job-logs"></a>

In [ ]:
endpoint = "jobs/{}/logs".format(job_response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

# Inference-методы ([содержание](#toc))<a class="anchor" id="inference"></a>

## Получить список сервисов ([содержание](#toc))<a class="anchor" id="inference-get-services-list"></a>

In [ ]:
endpoint = "inference/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json = {
    "services": response_json
}

In [ ]:
JSON(response_json)

## Получить информацию по сервису ([содержание](#toc))<a class="anchor" id="inference-get-service-info"></a>

In [ ]:
endpoint = "inference/v1/{}".format(response_json["services"][2]["name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json = {
    "service": response_json
}

In [ ]:
JSON(response_json)

## Создать образ для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-create-an-image"></a>

In [ ]:
s3.upload_file(
    "/home/jovyan/requirements.txt",
    credentials_dict["workspaces"][0]["bucket_name"],
    "inference_folder/requirements.txt",
)

In [ ]:
s3.upload_file(
    "/home/jovyan/dummy_mirror_serving_script.py",
    credentials_dict["workspaces"][0]["bucket_name"],
    "inference_folder/dummy_mirror_serving_script.py",
)

In [ ]:
endpoint = "inference/build/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "base_image": "registry.aicloud.sbcp.ru/base/horovod-cuda10.0-tf1.15.0",
    "run_script": "dummy_mirror_serving_script.py",
    "requirements_path": "requirements.txt",
    "artifacts_directory": "{}/inference_folder/".format(credentials_dict["workspaces"][0]["bucket_name"]),
    "AWS_ACCESS_KEY_ID": credentials_dict["workspaces"][0]["access_key_id"],
    "AWS_SECRET_ACCESS_KEY": credentials_dict["workspaces"][0]["secret_access_key"],
    "S3_ENDPOINT": credentials_dict["workspaces"][0]["endpoint_url"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
JSON(response_json)

## Просмотреть статус сборки образа для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-image-creation-status"></a>

In [ ]:
endpoint = "service/jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
JSON(json.loads(response.text))

## Просмотреть логи сборки образа для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-image-creation-logs"></a>

In [ ]:
endpoint = "service/jobs/{}/logs".format(response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

## Создать деплой на основе собранного образа ([содержание](#toc))<a class="anchor" id="inference-create-service"></a>

In [ ]:
endpoint = "inference/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "image": response_json["image"],
    "resources": {
        "cpu": 1,
        "memory": 10,
        "gpu": "1"
    },
    "replicas": {
        "min": 1,
        "max": 2
    }
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
print(response_json["metadata"]["name"])

## Отправить запрос к созданному сервису ([содержание](#toc))<a class="anchor" id="inference-get-a-response-from-service"></a>

In [ ]:
endpoint = "inference/v1/predict/{}/{}/".format(response_json["metadata"]["name"], response_json["metadata"]["name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "instances": [
        {
            "image_link": "1.jpg"
        },
        {
            "image_link": "2.jpg"
        },
        {
            "image_link": "3.jpg"
        }
    ]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
JSON(json.loads(response.text))